<h1><b>Crawling & Scrawling Data</h1></b> 

In [ ]:
import bs4
from bs4 import BeautifulSoup  
import pandas as pd
import numpy as np
import requests 
import math
import re
import time

In [ ]:
def generate_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")    
    #Crawl-delay: 3
    time.sleep(3)
    links=soup.find_all("a")
    list_of_links=[]
    for i,a in enumerate(links):
        string=a.get('href')
        time.sleep(3)
        if ("/draft/NBA_" in string)  :
            suffix=string.replace("/draft","")
            list_of_links.append(url+suffix)  
            if "1970" in string:
                break                  
    return list_of_links        

In [ ]:
def get_properties(url):
    pk=[]
    team=[]
    names=[]
    college=[]
    seasons=[]
    games=[]
    minutes=[]
    points=[]
    ttlribounds=[]
    asists=[]
    FGper=[]
    threePper=[]
    FTper=[]
    MP=[]
    PntperGame=[]
    ttlriboundsperGame=[]
    asistsperGame=[]
    WS=[]
    WSminutes=[]
    BPM=[]
    vorp=[]
    player_links=[]
    years=[]
    response = requests.get(url)
    time.sleep(3)
    soup = BeautifulSoup(response.content, "html.parser")
    #the number of <tr> within <table> 
    rows=soup.table.tbody.find_all("tr")
    #it takes 2 rounds inorder to compleate the draft 
    #each year has diffarent number of players to pick from --> difarent number of <tr> each <table>
    rounds=math.floor((len(rows)/2)-1)     
    for i,r in enumerate(rows):
                years.append(int(url[47:51]))
                #at some years like 2001 there are missing rows, "Minnesota forfeited the 29th overall pick."
                if r.find("td",attrs={"class":"center"}):
                    continue
                if (i < rounds or i>rounds+1) and (r.find_all("a",attrs={"href":re.compile("player")})) :
                    player_links.append(url[:36]+r.find_all("a",attrs={"href":re.compile("player")})[0].get("href")) #players_links 
                    l = r.find_all("td")
                    pk.append(l[0].string)
                    team.append(l[1].string)
                    names.append(l[2].string)
                    college.append(l[3].string)
                    seasons.append(l[4].string)
                    games.append(l[5].string)
                    minutes.append(l[6].string)
                    points.append(l[7].string)
                    ttlribounds.append(l[8].string)
                    asists.append(l[9].string)
                    FGper.append(l[10].string)
                    threePper.append(l[11].string)
                    FTper.append(l[12].string)
                    MP.append(l[13].string)
                    PntperGame.append(l[14].string)
                    ttlriboundsperGame.append(l[15].string)
                    asistsperGame.append(l[16].string)
                    WS.append(l[17].string)
                    WSminutes.append(l[18].string)
                    BPM.append(l[19].string)
                    vorp.append(l[20].string)
                        
                                                 
    return  pk,team, names, college ,seasons ,games ,minutes ,points, ttlribounds, asists, FGper, threePper, FTper, MP, PntperGame, ttlriboundsperGame, asistsperGame, WS, WSminutes, BPM ,vorp,player_links,years

In [ ]:
def generate_player_data(df):
    
    players_links=[]
    players_names=[]
    players_height=[]
    players_weight=[]
    players_position=[]
    
    
    players_names.extend(df['Names'].tolist())
    players_links.extend(df['player page'].tolist())
    
        
    #all players positions
    for link in players_links:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, "html.parser")
        try1 = soup.find("strong",string=re.compile("Position:")).next.next.split("  ")[1].rstrip()
        players_position.append(try1)
        
        #Gets Height
        res1=soup.body.find_all("span")
        
        
        if ')' in res1[9].text:  #Some players don't have full data
            players_height.append("0")
            players_weight.append("0")
            continue
        
        players_height.append(res1[9].text)
        
        #Gets Weight
        res2=soup.body.find_all("span")
        players_weight.append(res2[10].text)
        
        
    #turns Height 6-5 to 6.5
    for i in range(len(players_height)):
        players_height[i] = players_height[i].replace("-", ".")

    #Creat new dataframe with data
    data=pd.DataFrame(list(zip(players_names, players_height, players_weight,players_position)), columns=["Names", "Height", "Weight","Position"])
    
    #clean Height & Weight
    data['Height'] = data['Height'].astype(float)
    data['Height'] = 30.01818181818182 * data['Height']
    data['Weight'] = data['Weight'].str.replace('lb', '')
    data['Weight'] = data['Weight'].astype(float)
    data['Weight'] = 0.453592 * data['Weight']
    
    
    #split positions
    new = data["Position"].str.split("and", n = 1, expand = True)
    data["First Position"]= new[0]
    #data["Second Position"]= new[1]


    #Cleans first position from ',' or 'and'
    new2 = data["First Position"].str.split(",", n = 1, expand = True)
    data["First Position"]= new2[0]
    
    #Cleans first position from ' '
    data["First Position"] = data["First Position"].str.strip()
    
    #Merge with original data
    df_final = pd.merge(df, data, how='inner', on = 'Names')
    
    
    return df_final


In [ ]:
def clean_zero_pages_names(df):
    df = df[df['Height'] != 0]
    df = df.drop(['player page','Names','Position'], axis=1)
    
    
    return df

<h2><b> crawling links </b></h2>

Robots.txt request a 3 sec crawl-delay and the mount of Data thet we needed for our project therefore the Data collection process can take nearly an hour!

In [ ]:
links=[]
links.extend(generate_links("https://www.basketball-reference.com/draft"))

<b> collecting properties and arrange in DataFrame </b>

In [ ]:
df = pd.DataFrame()
df2 = pd.DataFrame()
for i,link in enumerate(links) :
    pk,team, names, college ,seasons ,games ,minutes ,points, ttlribounds, asists, FGper, threePper, FTper, MP, PntperGame, ttlriboundsperGame, asistsperGame, WS, WSminutes, BPM ,vorp,player_links,years = get_properties(link)
    df=pd.DataFrame(list(zip(pk,team,player_links,years,names, college,seasons ,games ,minutes ,points, ttlribounds, asists, FGper, threePper, FTper, MP, PntperGame, ttlriboundsperGame, asistsperGame, WS, WSminutes, BPM ,vorp)), columns=['Overall Pick','Teams','player page','year', 'Names','College','Seasons in NBA', '# Games','Minutes Played','Points','Total Rebounds','Assists','Field Goal %','3 Point Field Goal %', 'Free Throw %', 'Minutes Played per Game','Points per Game', 'Total Rebounds per Game', 'Assists per Game','Win Shares','Win Shares per 48 Minutes','Box +-','VORP'])
    df2=pd.concat([df2,df],axis=0)

<b>the pre-processed DataFrame</b>

<b>cleanning and processing Data</b>

In [ ]:
df2.dropna(axis=0,inplace=True)
df3=generate_player_data(df2.copy())
df4 = clean_zero_pages_names(df3.copy())

In [ ]:
df4.to_csv("machine_learning_data.csv")

<h3><b>collecting salaries</b></h3>

<p>inorder to collect salaries from the datasource wi nedd to crawl each player_page , searching for the first year at the salaries table in it. we vlidate that the salary is ctually the first one by matching the draft year from dataframe and the salary year from player_page </p>

In [ ]:
def generate_first_salary(df):
    salaries=[]
    for i,row in df.iterrows():
        response = requests.get(row[2])
        time.sleep(3)
        soup = BeautifulSoup(response.content, "html.parser")      
        l=soup.find_all("div")
        time.sleep(3)
        li=str(l[0]).split("<tr >")
        for k in li:
                flag=0
                if (str(row[3]) in k) and ("csk=" in k):
                    for i in k.split("csk=",2)[len(k.split("csk=",2))-1][:-20].split():
                        if i.strip('""').isdigit() :
                            num=int(i.strip('""'))
                            if num>100000:
                                salaries.append(num)
                                flag=1
                            break
                    break
        if flag==0:
            salaries.append(float("NaN"))       
    return salaries

In [ ]:
li=[]
li.extend(generate_first_salary(df3.copy()))

In [ ]:
df5["salary"]=li

In [ ]:
df5

In [ ]:
df5.dropna(axis=0).to_csv("Full_Data_with_salaries.csv")